In [1]:
%pylab inline
%matplotlib inline

# Global Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import os
import sys
import pickle
from pprint import pprint
from time import time
import datetime
from time import gmtime, strftime
import statsmodels.api as sm
from patsy import dmatrices

# Scikit-Learn imports
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split

from sklearn.cross_validation import cross_val_score
from sklearn import metrics

# Local Imports
path = str(os.path.expanduser('~')) + '/git/predictEPL/config'
sys.path.append(path)
import paths

sys.path.append(paths.UTILS)
import useful_methods

Populating the interactive namespace from numpy and matplotlib


## Data Manipulation Functions

In [2]:
# Definings
RESULT_FILE_NAME = "log_hash_all_ht.csv"

In [4]:
def ReadHashDf():
    # Read game_infos as df
    dfGameInfo = useful_methods.csv_dic_df(paths.READ_PATH_GAME_INFO + 'game_infos.csv')
    dfGameInfo = useful_methods.DropNanGames(dfGameInfo).copy().reset_index(drop=True)
    dfGameInfo.GW = [int(gw) for gw in dfGameInfo.GW]
    dfGameInfo = dfGameInfo.sort_values(['GW', 'away_team'], ascending=[True, True]).copy().reset_index(drop=True)

    # Read Hash Emolex Model result
    df = useful_methods.csv_dic_df(paths.READ_PATH_RESULTS + RESULT_FILE_NAME)
    df.GW = [int(gw) for gw in df.GW]
    df = df.sort_values(['GW', 'away_team'], ascending=[True, True]).copy().reset_index(drop=True)


    # Combine 2 dfs
    df['score_ht_away'] = [int(item) for item in dfGameInfo.score_ht_away]
    df['score_ht_home'] = [int(item) for item in dfGameInfo.score_ht_home]
    df['score_ft_away'] = [int(item) for item in dfGameInfo.score_ft_away]
    df['score_ft_home'] = [int(item) for item in dfGameInfo.score_ft_home]

    df['pn_away_neg'] = [float(item) for item in df.pn_away_neg]
    df['pn_away_pos'] = [float(item) for item in df.pn_away_pos]
    df['pn_home_neg'] = [float(item) for item in df.pn_home_neg]
    df['pn_home_pos'] = [float(item) for item in df.pn_home_pos]

    # 'home_win': 1, 'away_win': 0, 'draw': 2
    def Labeling(goal_diff):
        if goal_diff > 0:
            return 1
        elif goal_diff < 0:
            return 0
        else:
            return 2

    df['goal_diff_ht'] = df.score_ht_home - df.score_ht_away
    df['goal_diff_ft'] = df.score_ft_home - df.score_ft_away
    df['result'] = [Labeling(item) for item in df.goal_diff_ft]


    # **********************************************
    # Add previous 4 games points sum
    dfBe4GameSum = useful_methods.csv_dic_df(paths.DATA_HOME + "EPL/info_before_4game_sum.csv")

    team_homes = list(df.home_team)
    team_aways = list(df.away_team)
    GWs = [int(gw) for gw in df.GW]

    be_4game_sum_home = []
    be_4game_sum_away = []

    for index in range(len(GWs)):
        team_home = team_homes[index]
        team_away = team_aways[index]
        gw = GWs[index]

        be_4game_sum_home.append(int(dfBe4GameSum[dfBe4GameSum.team == team_home][str(gw)]))
        be_4game_sum_away.append(int(dfBe4GameSum[dfBe4GameSum.team == team_away][str(gw)]))

    df['be_4game_sum_away'] = be_4game_sum_away
    df['be_4game_sum_home'] = be_4game_sum_home    

    return df

In [6]:
# Create df for models.
def CreateDfForModel(ht_draw=False, ft_wld=False):
    df = ReadHashDf()
        
    # only for Win or Lose
    if not ft_wld:
        df = df[df.result != 2].copy().reset_index(drop=True)
    
    # HT: Equal
    if ht_draw:
        df = df[df.goal_diff_ht == 0].copy().reset_index(drop=True)
    
    dta = pd.DataFrame()
    
    # Teams
    dta['team_home'] = df.home_team
    dta['team_away'] = df.away_team

    # POS, NEG scores by percentage %
    dta['pn_home_pos'] = df.pn_home_pos / (df.pn_home_pos + df.pn_home_neg)
    dta['pn_home_neg'] = df.pn_home_neg / (df.pn_home_pos + df.pn_home_neg)
    dta['pn_away_pos'] = df.pn_away_pos / (df.pn_away_pos + df.pn_away_neg)
    dta['pn_away_neg'] = df.pn_away_neg / (df.pn_away_pos + df.pn_away_neg)
    
    dta['pn_diff_pos'] = dta['pn_home_pos'] - dta['pn_away_pos']
    
    # HF scores
    dta['score_ht_home'] = df.score_ht_home
    dta['score_ht_away'] = df.score_ht_away
    
    dta['goal_diff_ht'] = df['goal_diff_ht']

    # 'home_win': 1, 'away_win': 0, 'draw': 2
    dta['result'] = df.result
    
    # be_4game_sum
    dta['be_4game_sum_home'] = df.be_4game_sum_home / 12.0
    dta['be_4game_sum_away'] = df.be_4game_sum_away / 12.0
    
    return dta

In [7]:
# X: df, y: list
def CreateXy(df, team_name=False, hash_emolex=True, score_ht=True, be_4game_sum=False):
    if team_name and hash_emolex and score_ht and be_4game_sum:
        y, X = dmatrices('result ~ \
            be_4game_sum_home + be_4game_sum_away + \
            score_ht_home + score_ht_away + \
            pn_home_pos + pn_home_neg + pn_away_pos + pn_away_neg + \
            pn_diff_pos + \
            C(team_home) + C(team_away)',
            df, return_type="dataframe")
    elif hash_emolex and score_ht and be_4game_sum:
        y, X = dmatrices('result ~ \
            be_4game_sum_home + be_4game_sum_away + \
            score_ht_home + score_ht_away + \
            pn_home_pos + pn_home_neg + pn_away_pos + pn_away_neg + \
            pn_diff_pos',
            df, return_type="dataframe")
    elif hash_emolex and be_4game_sum:
        y, X = dmatrices('result ~ \
            be_4game_sum_home + be_4game_sum_away + \
            pn_home_pos + pn_home_neg + pn_away_pos + pn_away_neg + \
            pn_diff_pos',
            df, return_type="dataframe")
    elif score_ht and be_4game_sum:
        y, X = dmatrices('result ~ \
            score_ht_home + score_ht_away + \
            be_4game_sum_home + be_4game_sum_away',
            df, return_type="dataframe")
    elif hash_emolex:
        y, X = dmatrices('result ~ \
            pn_home_pos + pn_home_neg + pn_away_pos + pn_away_neg + \
            pn_diff_pos',
            df, return_type="dataframe")
    else:
        y, X = dmatrices('result ~ \
            score_ht_home + score_ht_away',
            df, return_type="dataframe")
        
    # flatten y into a 1-D array
    y = np.ravel(y)
    
    return X, y

## Models: Logistic, SVC

In [8]:
def ModelScoresWL(df, team_name, hash_emolex, score_ht, be_4game_sum, variable_scores=False):
    # instantiate a logistic regression model, and fit with X and y
    X, y = CreateXy(df, team_name, hash_emolex, score_ht, be_4game_sum)

    # Set Models
    model_log = LogisticRegression()
    model_svc = SVC()

    # Fit to models
    model_log = model_log.fit(X, y)
    model_svc = model_svc.fit(X, y)

    # Cross Validation
    scores_log = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=18)
    scores_svc = cross_val_score(SVC(), X, y, scoring='accuracy', cv=18)

    # Accuracy scores
    print("--------------------------\n")
    print("[Log]: \t%.3f (cv: %.3f)" % (model_log.score(X, y), scores_log.mean()))
    print("[SVC]: \t%.3f (cv: %.3f)" % (model_svc.score(X, y), scores_svc.mean()))
    
    
    if variable_scores:
        # Variable Scores
        dfVars = pd.DataFrame()
        dfVars['var'] = X.columns
        dfVars['score'] = np.transpose(model_log.coef_)
        print("\n", dfVars)

In [9]:
def ModelScoresWLD(df, team_name, hash_emolex, score_ht, be_4game_sum):
    # instantiate a logistic regression model, and fit with X and y
    X, y = CreateXy(df, team_name, hash_emolex, score_ht, be_4game_sum)

    # Set Models
    model_log = LogisticRegression()
    model_svc = SVC()

    # Fit to models
    model_log = model_log.fit(X, y)
    model_svc = model_svc.fit(X, y)

    # Cross Validation
    scores_log = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=18)
    scores_svc = cross_val_score(SVC(), X, y, scoring='accuracy', cv=18)

    # Accuracy scores
    print("--------------------------\n")
    print("[Log]: \t%.3f (cv: %.3f)" % (model_log.score(X, y), scores_log.mean()))
    print("[SVC]: \t%.3f (cv: %.3f)" % (model_svc.score(X, y), scores_svc.mean()))

### Accuracy

In [10]:
# Create DFs for models
# WL: win, lose; WLD: win, lose, draw
dfWL = CreateDfForModel(ht_draw=False, ft_wld=False)
dfWLD = CreateDfForModel(ht_draw=False, ft_wld=True)

In [12]:
# Win, Lose
ModelScoresWL(dfWL, team_name=False, hash_emolex=False, score_ht=True, be_4game_sum=False, variable_scores=True)
ModelScoresWL(dfWL, team_name=False, hash_emolex=False, score_ht=True, be_4game_sum=True, variable_scores=True)
ModelScoresWL(dfWL, team_name=False, hash_emolex=True, score_ht=False, be_4game_sum=False, variable_scores=True)
ModelScoresWL(dfWL, team_name=False, hash_emolex=True, score_ht=False, be_4game_sum=True, variable_scores=True)
ModelScoresWL(dfWL, team_name=False, hash_emolex=True, score_ht=True, be_4game_sum=True, variable_scores=True)
ModelScoresWL(dfWL, team_name=True, hash_emolex=True, score_ht=True, be_4game_sum=True, variable_scores=False)

--------------------------

[Log]: 	0.766 (cv: 0.765)
[SVC]: 	0.766 (cv: 0.765)

              var     score
0      Intercept  0.107486
1  score_ht_home  1.320774
2  score_ht_away -1.762310
--------------------------

[Log]: 	0.781 (cv: 0.743)
[SVC]: 	0.766 (cv: 0.765)

                  var     score
0          Intercept  0.235985
1      score_ht_home  1.309614
2      score_ht_away -1.734518
3  be_4game_sum_home -0.092961
4  be_4game_sum_away -0.485873
--------------------------

[Log]: 	0.695 (cv: 0.682)
[SVC]: 	0.648 (cv: 0.634)

            var     score
0    Intercept  0.105339
1  pn_home_pos  0.836647
2  pn_home_neg -0.731308
3  pn_away_pos -0.855293
4  pn_away_neg  0.960632
5  pn_diff_pos  1.691940
--------------------------

[Log]: 	0.703 (cv: 0.686)
[SVC]: 	0.602 (cv: 0.595)

                  var     score
0          Intercept  0.263121
1  be_4game_sum_home -0.181918
2  be_4game_sum_away -0.870859
3        pn_home_pos  0.970561
4        pn_home_neg -0.707440
5        pn_away_

In [13]:
ModelScoresWLD(dfWLD, team_name=False, hash_emolex=False, score_ht=True, be_4game_sum=False)
ModelScoresWLD(dfWLD, team_name=False, hash_emolex=False, score_ht=True, be_4game_sum=True)
ModelScoresWLD(dfWLD, team_name=False, hash_emolex=True, score_ht=False, be_4game_sum=False)
ModelScoresWLD(dfWLD, team_name=False, hash_emolex=True, score_ht=False, be_4game_sum=True)
ModelScoresWLD(dfWLD, team_name=False, hash_emolex=True, score_ht=True, be_4game_sum=True)
ModelScoresWLD(dfWLD, team_name=True, hash_emolex=True, score_ht=True, be_4game_sum=True)

--------------------------

[Log]: 	0.554 (cv: 0.553)
[SVC]: 	0.582 (cv: 0.493)
--------------------------

[Log]: 	0.588 (cv: 0.541)
[SVC]: 	0.599 (cv: 0.498)
--------------------------

[Log]: 	0.497 (cv: 0.499)
[SVC]: 	0.469 (cv: 0.459)
--------------------------

[Log]: 	0.508 (cv: 0.502)
[SVC]: 	0.435 (cv: 0.431)
--------------------------

[Log]: 	0.571 (cv: 0.526)
[SVC]: 	0.588 (cv: 0.542)
--------------------------

[Log]: 	0.689 (cv: 0.507)
[SVC]: 	0.559 (cv: 0.542)
